In [ ]:
import vtk
import pathlib


def export_polydata_with_cell_data(polydata, cell_data_array_name, output_filename):
    new_polydata = vtk.vtkPolyData()
    new_polydata.DeepCopy(polydata)

    cell_arrays = [
        new_polydata.GetCellData().GetArrayName(i)
        for i in range(new_polydata.GetCellData().GetNumberOfArrays())
    ]
    arrays_to_remove = [arr for arr in cell_arrays if arr != cell_data_array_name]

    # Copy the cell data array
    cell_data = new_polydata.GetCellData()
    if cell_data:
        print("Original number of arrays: ", cell_data.GetNumberOfArrays())
        for array_name in arrays_to_remove:
            cell_data.RemoveArray(array_name)

        # Write the polydata to file
        print(new_polydata.GetCellData().GetNumberOfArrays())
        writer = vtk.vtkXMLPolyDataWriter()
        writer.SetFileName(output_filename)
        writer.SetInputData(new_polydata)
        writer.SetDataModeToAscii()
        writer.Write()
    else:
        print("Cell data array '{}' not found in the polydata.".format(cell_data_array_name))

In [ ]:
# source_path = pathlib.Path("/mnt/disk01/insight-volume/V0/Eztec")
source_path = pathlib.Path("/mnt/disk01/insight-volume/V0/PrologisCajamar4")

# subproject_name = "G100"
# subproject_name = "G200"
subproject_name = "G300"
# subproject_name = "building"

# 293 not found
# for direction in ["000", "023", "045", "068", "090", "113", "135", "158", "180", "203", "225", "270", "315", "338"]:
# for direction in ["000", "023", "045", "068", "090", "113", "135", "158", "180", "203", "225", "248", "270", "293", "315", "338"]:
for direction in ["000", "045", "090", "135", "180", "225", "270", "315"]:
    # for direction in ["013", "058", "103", "148", "193", "238", "283", "328"]:

    # Path to the input XML polydata file
    # file_folder = source_path / subproject_name / "cases" / direction / "cp_root" / "raw"
    file_folder = (
        source_path / subproject_name / "cases" / direction / "cp_root" / "raw" / "cp" / "peak_rho"
    )
    # file_folder = source_path / subproject_name / "cases" / direction / "Cf_floors_y" / "raw"
    output_folder = source_path / subproject_name / "cases" / direction / "cp_root"
    input_filename = file_folder / "cp.stats.vtp"
    # input_filename = file_folder / "Cfy.stats.vtp"

    # Create a reader
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_filename)
    reader.Update()

    # Get the polydata
    polydata = reader.GetOutput()

    # Iterate over cell data arrays
    cell_data = polydata.GetCellData()
    if cell_data:
        num_arrays = cell_data.GetNumberOfArrays()
        for i in range(num_arrays):
            array_name = cell_data.GetArrayName(i)
            # print(array_name)
            # stat_name = "_".join(array_name.split("_")[1:])
            print("Exporting: ", array_name)
            output_filename = output_folder / f"{array_name}.vtp"
            export_polydata_with_cell_data(polydata, array_name, output_filename)
    else:
        print("No cell data arrays found in the polydata.")